# TV Grid Scrapper
Python Version 3.11.4  
`pip install requests bs4`

In [86]:
from requests import get as get_url
from bs4 import BeautifulSoup
from dataclasses import dataclass
from json import dumps as json_dumps
from IPython.core.display import display_json, display_html

CAPTURE_URL = 'https://www.programme-tv.net/programme/programme-tnt.html'

In [87]:
TVGRID_PAGE = get_url(CAPTURE_URL)
soup = BeautifulSoup(TVGRID_PAGE.content, 'html.parser')
TVGRID_RAW_BLOCKS = soup.select('.mainBroadcastCard-infos')

In [88]:
@dataclass
class Channel:
    channel_number: str
    channel_name: str

channels = []

for channel in soup.select('.gridRow-cardsChannelName'):
    channel_number = channel.select_one('.sr-only').text.strip().replace('N°', '') # Numéro de la chaine
    channel_name = channel.text.strip().replace(f'N°{channel_number}', '') # Description de la chaîne
    channels.append(Channel(channel_number=channel_number, channel_name=channel_name))

## Render data as JSON

In [89]:
@dataclass
class Program:
    startBroadcasting: str
    title: str
    subtitle: str
    duration_hours: str
    duration_minutes: str
    is_broadcast_new: bool
    channel: Channel

programs = []

channel_counter = 0
for programme in TVGRID_RAW_BLOCKS:
    startBroadcasting = programme.select_one('.mainBroadcastCard-startingHour')
    if startBroadcasting:
        startBroadcasting = startBroadcasting.text.strip()

    title = programme.select_one('.mainBroadcastCard-title')
    if title:
        title = title.text.strip()

    subtitle = programme.select_one('.mainBroadcastCard-subtitle')
    if subtitle:
        subtitle = subtitle.text.strip()

    duration = programme.select_one('.mainBroadcastCard-durationContent')
    if duration:
        duration = duration.text.strip()
        duration = duration.replace('min', '')
        if (duration.count('h')):
            duration_hours = duration.split('h')[0]
            if duration.split('h')[1]:
                duration_minutes = duration.split('h')[1]
            else:
                duration_minutes = '0'
        else:
            duration_hours = '0'
            duration_minutes = duration
    
    broadcast_new = programme.select_one('.mainBroadcastCard-new')
    if broadcast_new:
        broadcast_new = broadcast_new.text.strip()
        if broadcast_new == 'Inédit':
            is_broadcast_new = True
        else:
            is_broadcast_new = False
    else:
        is_broadcast_new = False

    programs.append(Program(startBroadcasting=startBroadcasting,
                            title=title,
                            subtitle=subtitle,
                            duration_hours=duration_hours,
                            duration_minutes=duration_minutes,
                            is_broadcast_new=is_broadcast_new,
                            channel=channels[int(channel_counter)]))
    channel_counter += 0.5

dump = json_dumps(programs, default=lambda o: o.__dict__, ensure_ascii=False)
display_json(dump, raw=True)

## Render data as HTML

In [91]:
raw_html = '''<h1>Les programmes de ce soir</h1>
<style>
.card_broadcast {
background-color: lightgrey;
}
.card_program_odd {
background-color: lightgreen;
}
.card_program_even {
background-color: lightblue;
}
.broadcast_start {
font-weight: bold;
}
.duration {
font-weight: bold;
}
.first_broadcast {
display: flex;
font-weight: bold;
font-size: large;
text-transform: uppercase;
justify-content: flex-end;
}
</style>
'''
odd_line = True
for program in programs:
    if odd_line:
        raw_html = raw_html + f'<div class="card_broadcast"><h2>{program.channel.channel_name} (TNT {program.channel.channel_number})</h2>'
    
    if odd_line:
        raw_html = raw_html + f'<div class="card_program_odd">'
    else:
        raw_html = raw_html + f'<div class="card_program_even">'

    raw_html = raw_html + f'<h3>{program.title}</h3>'

    if program.subtitle:
        raw_html = raw_html + f'<h4>{program.subtitle}</h4>'
    
    raw_html = raw_html + f'<p>Départ <span class="broadcast_start">{program.startBroadcasting}</span>, durée <span class="duration">{program.duration_hours}h{program.duration_minutes}</span></p>'
    
    if program.is_broadcast_new:
        raw_html = raw_html + '<p class="first_broadcast">inédit</p>'
    raw_html = raw_html + f'</div>'

    if not odd_line:
        raw_html = raw_html + f'</div>'
    odd_line = not odd_line

display_html(raw_html, raw=True)

Les programmes de ce soir 

 TF1 (TNT 1) 30 ans d'émissions cultes La saga des jeux TV Départ 21h10 , durée 2h30 inédit 30 ans d'émissions cultes Les plus grands fous rires de la télé Départ 23h40 , durée 2h10 France 2 (TNT 2) Fort Boyard Départ 21h10 , durée 2h10 inédit Fort Boyard : toujours plus fort ! Départ 23h20 , durée 0h59 France 3 (TNT 3) Police de caractères Un loup dans la bergerie Départ 21h10 , durée 1h35 Meurtres à... Meurtres à Blois Départ 22h45 , durée 1h35 Canal+ (TNT 4) Maestro(s) Départ 21h05 , durée 1h26 Le Parfum vert Départ 22h31 , durée 1h39 France 5 (TNT 5) Echappées belles Sur les sentiers varois Départ 21h00 , durée 1h35 Echappées belles Spéciale Martinique gourmande Départ 22h35 , durée 1h30 M6 (TNT 6) NCIS Echange de prisonniers Départ 21h10 , durée 0h45 inédit NCIS Croisière d'enfer Départ 21h55 , durée 0h55 Arte (TNT 7) Serengeti, la grande cavalcade des animaux Luttes de pouvoir Départ 20h50 , durée 0h45 inédit Secrets de chats Départ 23h05 , durée 0h55 C8 (TNT 8) Compromis Départ 21h10 , durée 1h42 Amis Départ 22h52 , durée 1h35 W9 (TNT 9) La petite histoire de France Départ 21h05 , durée 1h55 La petite histoire de France Départ 23h00 , durée 1h45 TMC (TNT 10) Columbo Une ville fatale Départ 20h55 , durée 1h30 Le transporteur : l'héritage Départ 22h25 , durée 1h50 TFX (TNT 11) Chroniques criminelles Départ 21h05 , durée 1h45 Chroniques criminelles Départ 22h50 , durée 1h55 NRJ 12 (TNT 12) Big Bang Theory La Conjecture du Batbocal Départ 21h10 , durée 0h30 Big Bang Theory Réaction ! Départ 22h30 , durée 0h30 La Chaîne parlementaire (TNT 13) L'odyssée des plages Départ 21h00 , durée 1h58 Corée du Nord, un plan pour survivre Départ 22h58 , durée 1h32 France 4 (TNT 14) Le Requiem de Verdi Départ 21h10 , durée 1h35 Carmina Burana Départ 22h45 , durée 1h09 BFMTV (TNT 15) 7 Jours BFM Départ 20h00 , durée 2h0 Week-end Direct Départ 22h00 , durée 2h0 CNEWS (TNT 16) Soir Info Soir Info Départ 20h59 , durée 2h28 Journal Départ 23h29 , durée 0h28 CSTAR (TNT 17) Enquêtes paranormales Episode 20 Départ 21h10 , durée 1h35 Enquêtes paranormales Episode 16 Départ 22h45 , durée 1h25 Gulli (TNT 18) Total Wipeout Made In USA Vers de terre et bubble tea Départ 21h05 , durée 0h55 Total Wipeout Made In USA Ouvrez le dico ! Départ 22h00 , durée 0h50 TF1 Séries Films (TNT 20) Joséphine, ange gardien Joséphine fait de la résistance Départ 21h00 , durée 2h05 Joséphine, ange gardien Suivez le guide Départ 23h05 , durée 2h0 L'Equipe (TNT 21) Para-athlétisme : Championnats du monde Championnats du monde. 7e jour. Départ 18h05 , durée 3h30 Monster Jam 5e à 8e manches. Départ 21h35 , durée 1h50 6ter (TNT 22) Vive le camping Porto : au cœur du camping le moins cher de Corse ! Départ 21h05 , durée 2h15 Vive le camping Chez les Ch'tis Départ 23h20 , durée 2h0 RMC Story (TNT 23) Retour à l'instinct primaire Piqués par la jungle Départ 21h10 , durée 0h55 Retour à l'instinct primaire Coup de chaleur au Mexique Départ 22h05 , durée 0h55 RMC Découverte (TNT 24) Europapark : la techno des manèges à sensation Départ 21h10 , durée 0h57 Roller Coaster : une technologie à sensations Départ 22h07 , durée 0h56 Chérie 25 (TNT 25) Jeanne Poisson, marquise de Pompadour Départ 21h05 , durée 1h55 Héritages Spéciale Saint-Valentin : de l'amour à la guerre Départ 23h00 , durée 1h40 LCI - La Chaîne Info (TNT 26) Le 20H Darius Rochebin Départ 20h00 , durée 2h0 22H Darius Rochebin Départ 22h00 , durée 2h0 Franceinfo (TNT 27) Le doc de l'info Départ 20h03 , durée 0h26 Tour de France 2023 Départ 20h35 , durée 0h12 Paris Première (TNT 41) Michaël Gregorio : j'ai 10 ans Départ 21h00 , durée 2h10 Michaël Gregorio pirate les chanteurs Départ 23h10 , durée 2h05 Canal+ Sport (TNT 42) IndyCar Series : Grand Prix de Toronto Grand Prix de Toronto. Essais qualificatifs. IndyCar Series. Départ 20h45 , durée 1h36 Golf : Open de Nicholasville Open de Nicholasville. 3e tour. Circuit américain. Départ 22h21 , durée 2h38 Canal+ Cinéma (TNT 43) Un talent en or massif D